## Análisis de sentimientos:clasificador de reseñas de la tienda de Google Play (NAIVE BAYES ALGORITHM)

### Paso 1: Carga del conjunto de datos

In [1]:
import pandas as pd

total_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv")

total_data.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


En base a las instrucciones del proyecto, sabemos que, en este conjunto de datos encontrarás las siguientes variables:

+ package_name: Nombre de la aplicación móvil, del tipo categórico
+ review: Comentario sobre la aplicación móvil, del tipo categórico
+ polarity: Variable de clase (0 o 1), siendo 0 un comentario negativo y 1, positivo, del tipo numérico

### Paso 2: Estudio de variables y su contenido

Sabemos por instrucciones del proyecto que tenemos solo 3 variables: 2 predictoras y una etiqueta dicotómica. De las dos predictoras, realmente solo nos interesa la parte del comentario, ya que el hecho de clasificar un comentario en positivo o negativo dependerá de su contenido, no de la aplicación de la que se haya escrito. Por lo tanto, la variable `package_name` habría que eliminarla.

Procedamos a eliminar espacios y convertir a minúsculas el texto:

In [2]:
def apply_preprocess(df):
    df = df.drop("package_name", axis=1)
    df["review"] = df["review"].str.strip().str.lower()

    return df

total_data = apply_preprocess(total_data)

total_data.head()

,review,polarity
0,privacy at least put some option appear offlin...,0
1,"messenger issues ever since the last update, i...",0
2,profile any time my wife or anybody has more t...,0
3,the new features suck for those of us who don'...,0
4,forced reload on uploading pic on replying com...,0


Dividir el conjunto de datos en entrenamiento y prueba : X_train, X_test, y_train, y_test

In [3]:
from sklearn.model_selection import train_test_split

X = total_data["review"]
y = total_data["polarity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train.head()

331    just did the latest update on viber and yet ag...
733    keeps crashing it only works well in extreme d...
382    the fail boat has arrived the 6.0 version is t...
704    superfast, just as i remember it ! opera mini ...
813    installed and immediately deleted this crap i ...
Name: review, dtype: object

Paseos ahota a transformar el texto en una matriz de recuento de palabras. Esta es una forma de obtener características numéricas a partir del texto. Para ello, utilizamos el conjunto de train para entrenar el transformador y la aplicamos en test:

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vec_model = CountVectorizer(stop_words = "english")
X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()

X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Paso 3: Construye un naive bayes

Dada la  naturaleza de la variable respuesta (variable binaria) y las variables predictoras (categorias), debemos escoger el MultinomialNB.

Procedamos a realizar la carga y entrenamiento del modelo.

In [5]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

Una vez entrenado el modelo, pasamos al proceso de prediccion y evaluacion de la prediccion.

In [6]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0])

In [7]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.8156424581005587

Este resultado indica que el modelo fue capaz de predecir el 82% de la dara de prueba, por lo cual el modelo es adecuado.

Probemos ahora, hacer las pruebas con las dos otras implementaciones Gaussiana y Bernoulli, y confirmemos si el modelo anterior es el adecuado en contrastes con los dos previamente indicados aqui.

Para ello es suficiente entrenar, evaluar y observar el accuracy score

In [9]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB

U = [GaussianNB(), BernoulliNB()]

for model_aux in U:
    model_aux.fit(X_train, y_train)
    y_pred_aux = model_aux.predict(X_test)
    print(f"{model_aux} with accuracy: {accuracy_score(y_test, y_pred_aux)}")

GaussianNB() with accuracy: 0.8044692737430168
BernoulliNB() with accuracy: 0.770949720670391


El resultado de los tres modelos son relativamente similares, siendo mejor el obtenido con suspuesto Multinomial, seguido por el modelo Gaussiano y el modelo Bernoulli , sin embargo, señalamos que:

+ Naive Bayes Gaussiano, esta diseñado para datos continuos, lo cual no se corresponde a la naturaleza de nuestros datos de estudios.
+ Naive Bayes Bernoulli, esta diseñado para datos binarios, lo cual no se corresponde a la naturaleza de nuestros datos de estudios.

En base a lo anterior indicado, se reafirma que, el modelo que debe ser considerado es el Naive Bayes con supuesto Multinomial.

### Paso 4: Optimiza el modelo anterior

Procedamos el proceso de optimizacion usando el metodo de busqueda de cuadriculas para los parametros Hiperparámetros:

+ alpha: Este parámetro ayuda a evitar problemas de cero probabilidades cuando una palabra no aparece en una clase durante el entrenamiento.

+ fit_prior: Este parámetro determina si se calculan las probabilidades a priori de las clases a partir de los datos de entrenamiento. Si se establece en False, se asume una distribución uniforme de las clases.

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# Definimos los valores que queremos probar para cada hiperparámetro
hyperparams = {
    'alpha': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'fit_prior': [True, False]
}

# Realizamos la búsqueda por cuadrícula
grid_search = GridSearchCV(model, hyperparams, cv=15)
grid_search
grid_search.fit(X_train, y_train)


GridSearchCV(cv=15, estimator=MultinomialNB(),
             param_grid={'alpha': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6,
                                   0.7, 0.8, 0.9, 1.0],
                         'fit_prior': [True, False]})

Ahora identifiquemos el valor de los mejores parametros

In [44]:
# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(f"Lo mejores hiperparametros son: {best_params}")

Lo mejores hiperparametros son: {'alpha': 0.5, 'fit_prior': True}


Realizamos ahora el entrenamiento del modelo

In [15]:
# Entrenamos el modelo con los mejores parámetros
best_model = MultinomialNB(**best_params)
best_model.fit(X_train, y_train)

MultinomialNB(alpha=0.5)

Una vez entrenado el modelo, pasamos al proceso de prediccion y evaluacion de la prediccion.

In [16]:
# Hacemos predicciones con el modelo óptimo
y_pred = best_model.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.8268156424581006

Este resultado indica que el modelo fue capaz de predecir el 83% de la dara de prueba, por lo cual el modelo es adecuado.

Adicionalmente, podemos señalar una mejora de alrededor de 1% en la capacidad de predecir respecto al modelo sin optimizar.


### Paso 5: Guarda el modelo


In [ ]:
import os

# Suponiendo que el directorio "my_models" está en la misma carpeta que su script
models_dir = "my_models"

# Crea el directorio si no existe
os.makedirs(models_dir, exist_ok=True)

from pickle import dump

# Guarde el modelo con la nueva ruta
dump(model, open(os.path.join(models_dir, "naive_bayes_alpha_1-9176382_fit_prior_False_42.sav"), "wb"))


### Paso 6: Explora otras alternativas

¿Qué otros modelos de los que hemos estudiado podrías utilizar para intentar superar los resultados de un Naive Bayes? 

Dado a que estamos realizando una clasificación, y tenemos tanto datos numericos como categoricos podemos utilizar un modelo de arbol, ya que su principal ventaja es su facil interpretacion y pueden manejar datos mixtos (numéricos y categóricos). Pero su principal desventaja es que puede ser propensos al sobreajuste, especialmente con árboles profundos.

En base a esto implementemos el arbol de clasificación:

In [19]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state = 42)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

Procedamos ahora al proceso de prediccion del modelo

In [24]:
y_pred = model.predict(X_test)
y_pred

array([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0])

Con los datos en crudo es muy complicado saber si el modelo está acertando o no. Para ello, debemos compararlo con la realidad. Existe una gran cantidad de métricas para medir la efectividad de un modelo a la hora de predecir, entre ellas la precisión (accuracy), que es la fracción de predicciones que el modelo realizó correctamente.

In [26]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7150837988826816

En este caso el modelo de arbol de clasificacion es capaz de predecir la data de prueba en un 72%

realicemos el proceso de optimizacion, a fin de evaluar si este arbol se puede mejorar

Realicemos esto mediante el metodo de la búsqueda en cuadrícula (grid search), los hiperparametros a considerar seran para este caso:

+ El indice de Gini y la entropia.
+ La profundidad o niveles de ramificacion del arbol
+ Division minima de las muestras.
+ Muestras minimas a tomar.

In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

hyperparams = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

grid = GridSearchCV(model, hyperparams, scoring = "accuracy", cv = 10)
grid

GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(criterion='entropy', max_depth=5,
                                              min_samples_leaf=4,
                                              random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy')

Ahora observemos cuales serian los mejores valores para los hiperametros 

In [43]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

grid.fit(X_train, y_train)

print(f"Lo mejores hiperparametros son: {grid.best_params_}")

Lo mejores hiperparametros son: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10}


Usemos la data de entrenamiento para ajustar el modelo con los mejores hiperparametros.

In [45]:
model = DecisionTreeClassifier(**grid.best_params_)
model.fit(X_train, y_train)

DecisionTreeClassifier(min_samples_split=10)

Una vez ajustado realicemos las predicciones pertinenes y luego comparemos predicciones con data de prueba

In [46]:
y_pred = model.predict(X_test)
y_pred

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0])

In [47]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7094972067039106

Esta solucion nos indica que el modelo de arbol sin y con la optimizacion de los parametros es comparable.

Por otro lado comparando con el modelo de Naive Bayes Multinomial, el modelo de arbol para este caso resulto ser inferior respecto a su capacidad de prediccion.